In [0]:
import dlt
from pyspark.sql import functions as F
from pyspark.sql.window import Window

@dlt.table(
    comment="Gold table driven by config table - config_silver_to_gold table"
)
def gold_table():
    # Read Silver table
    df = dlt.read("silver_table")

    # Drop null customerId
    df = df.filter(F.col("customerId").isNotNull())

    # Read column config table and filter active columns
    cfg = dlt.read("`unity-veersa`.config_tables.config_silver_to_gold").filter(F.col("is_active") == True)

    # Select only active columns from Silver and apply aliases
    select_exprs = [F.col(c["column_name"]).alias(c["alias_name"]) for c in cfg.collect()]
    df = df.select(*select_exprs)

    # Deduplicate: keep latest per customerId
    window_spec = Window.partitionBy("customerId").orderBy(F.col("ingest_ts").desc())
    df = df.withColumn("row_num", F.row_number().over(window_spec)) \
           .filter(F.col("row_num") == 1) \
           .drop("row_num")

    # Create derived columns
    df = df.withColumn(
        "address",
        F.concat_ws(", ", F.col("city"), F.col("state"), F.col("zip_prefix"))
    ).withColumn(
        "zip_first_digit", F.substring(F.col("zip_prefix"), 1, 1)
    ).withColumn(
        "region",
        F.when(F.col("zip_first_digit").isin("0","1"), "East")
         .when(F.col("zip_first_digit").isin("2","3"), "South")
         .when(F.col("zip_first_digit").isin("4","5"), "West")
         .when(F.col("zip_first_digit").isin("6","7"), "North")
         .otherwise("Central")
    ).withColumn(
        "record_source", F.lit("silver_table")
    )

    # Final selection: include all columns from config plus derived columns
    derived_cols = ["address", "region"]
    df_gold = df.select(*[c["alias_name"] for c in cfg.collect()] + derived_cols)

    df_gold = df_gold.drop("city", "state", "zip_prefix")

    return df_gold
